In [1]:
import os
import json
import glob
import cv2
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
### cocodata를 활용하여 각 파손별 및 train/test/val 별 마스크 생성하기 ###
### grayscale로 mask 생성 ###

import json

from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import cv2
import os
import random
import shutil
import numpy as np
from tqdm import tqdm
from PIL import Image

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

catego = ['Breakage', 'Crushed', 'Scratched', 'Separated']
img_origin = '../data/Dataset/1.원천데이터/damage/'
json_origin = '../data/Dataset/2.라벨링데이터/damage/'


for c in catego:
        part_json = '../data/datainfo/damage_' + c + '_train.json'
        test_json = '../data/datainfo/damage_' + c + '_test.json'
        val_json  = '../data/datainfo/damage_' + c + '_val.json'

        with open(part_json, 'r') as f:
                trpart = json.load(f)

        with open(test_json, 'r') as f:
                tepart = json.load(f)

        with open(val_json, 'r') as f:
                vapart = json.load(f)        

        coco = COCO(part_json)
        coco2 = COCO(test_json)
        coco3 = COCO(val_json)

        img_ids = coco.getImgIds()
        img_id2 = coco2.getImgIds()
        img_id3 = coco3.getImgIds()

        cate = {c : []}
        cate_color =  (random.randrange(0, 256), random.randrange(0, 256), random.randrange(0, 256))

        for i in tqdm(range(0, len(trpart['images']))):
                id = int(img_ids[i])
                image_name = trpart['images'][i]['file_name']
                if not os.path.exists('../data/Dataset/damage_type/' + c + '_img_train/') and not os.path.exists('../data/Dataset/damage_type/' + c + '_json_train/'):
                        os.makedirs('../data/Dataset/damage_type/' + c + '_img_train/')
                        os.makedirs('../data/Dataset/damage_type/' + c + '_json_train/')
                shutil.copyfile(img_origin + image_name, '../data/Dataset/damage_type/' + c + '_img_train/' + image_name)
                shutil.copyfile(json_origin + image_name.replace('.jpg', '.json'), '../data/Dataset/damage_type/' + c + '_json_train/' + image_name.replace('.jpg', '.json'))
                image_infos = coco.loadImgs(id)[0]
                width = image_infos['width']
                height = image_infos['height']

                for j in range(0, len(trpart['annotations'])):
                        if trpart['annotations'][j]['image_id'] == id:
                                if len(trpart['annotations'][j]['segmentation'][0]) > 1:
                                        for i in trpart['annotations'][j]['segmentation'][0]:
                                                if trpart['annotations'][j]['damage'] in cate.keys():
                                                        cate[trpart['annotations'][j]['damage']].append(i)
                                else:
                                        if trpart['annotations'][j]['damage'] in cate.keys():
                                                cate[trpart['annotations'][j]['damage']].append(trpart['annotations'][j]['segmentation'][0][0])

                mask = np.zeros((height,width, 3),dtype = np.uint8)
                if not os.path.exists('../data/Dataset/damage_mask/' + c + '_train_mask/'):
                        os.makedirs('../data/Dataset/damage_mask/' + c + '_train_mask/')
                for j in range(0, len(cate[c])):
                        mask = cv2.fillPoly(mask, [np.array(cate[c][j][0])], cate_color)
                cv2.imwrite(f'../data/Dataset/damage_mask/' + c + '_train_mask/' + image_name.replace('.jpg', '') + '.png', cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY))
                cate[c] = []
        
        for i in tqdm(range(0, len(tepart['images']))):
                id = int(img_id2[i])
                image_name = tepart['images'][i]['file_name']
                if not os.path.exists('../data/Dataset/damage_type/' + c + '_img_test/') and not os.path.exists('../data/Dataset/damage_type/' + c + '_json_test/'):
                        os.makedirs('../data/Dataset/damage_type/' + c + '_img_test/')
                        os.makedirs('../data/Dataset/damage_type/' + c + '_json_test/')
                shutil.copyfile(img_origin + image_name, '../data/Dataset/damage_type/' + c + '_img_test/' + image_name)
                shutil.copyfile(json_origin + image_name.replace('.jpg', '.json'), '../data/Dataset/damage_type/' + c + '_json_test/' + image_name.replace('.jpg', '.json'))
                image_infos = coco2.loadImgs(id)[0]
                width = image_infos['width']
                height = image_infos['height']

                for j in range(0, len(tepart['annotations'])):
                        if tepart['annotations'][j]['image_id'] == id:
                                if len(tepart['annotations'][j]['segmentation'][0]) > 1:
                                        for i in tepart['annotations'][j]['segmentation'][0]:
                                                if tepart['annotations'][j]['damage'] in cate.keys():
                                                        cate[tepart['annotations'][j]['damage']].append(i)
                                else:
                                        if tepart['annotations'][j]['damage'] in cate.keys():
                                                cate[tepart['annotations'][j]['damage']].append(tepart['annotations'][j]['segmentation'][0][0])

                mask = np.zeros((height,width, 3),dtype = np.uint8)
                if not os.path.exists('../data/Dataset/damage_mask/' + c + '_test_mask/'):
                        os.makedirs('../data/Dataset/damage_mask/' + c + '_test_mask/')
                for j in range(0, len(cate[c])):
                        mask = cv2.fillPoly(mask, [np.array(cate[c][j][0])], cate_color)
                cv2.imwrite(f'../data/Dataset/damage_mask/' + c + '_test_mask/' + image_name.replace('.jpg', '') + '.png', cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY))
                cate[c] = []

        for i in tqdm(range(0, len(vapart['images']))):
                id = int(img_id3[i])
                image_name = vapart['images'][i]['file_name']
                if not os.path.exists('../data/Dataset/damage_type/' + c + '_img_val/') and not os.path.exists('../data/Dataset/damage_type/' + c + '_json_val/'):
                        os.makedirs('../data/Dataset/damage_type/' + c + '_img_val/')\compute\tpus
                        os.makedirs('../data/Dataset/damage_type/' + c + '_json_val/')
                shutil.copyfile(img_origin + image_name, '../data/Dataset/damage_type/' + c + '_img_val/' + image_name)
                shutil.copyfile(json_origin + image_name.replace('.jpg', '.json'), '../data/Dataset/damage_type/' + c + '_json_val/' + image_name.replace('.jpg', '.json'))
                image_infos = coco3.loadImgs(id)[0]
                width = image_infos['width']
                height = image_infos['height']

                for j in range(0, len(vapart['annotations'])):
                        if vapart['annotations'][j]['image_id'] == id:
                                if len(vapart['annotations'][j]['segmentation'][0]) > 1:
                                        for i in vapart['annotations'][j]['segmentation'][0]:
                                                if vapart['annotations'][j]['damage'] in cate.keys():
                                                        cate[vapart['annotations'][j]['damage']].append(i)
                                else:
                                        if vapart['annotations'][j]['damage'] in cate.keys():
                                                cate[vapart['annotations'][j]['damage']].append(vapart['annotations'][j]['segmentation'][0][0])

                mask = np.zeros((height,width, 3),dtype = np.uint8)
                if not os.path.exists('../data/Dataset/damage_mask/' + c + '_val_mask/'):
                        os.makedirs('../data/Dataset/damage_mask/' + c + '_val_mask/')
                for j in range(0, len(cate[c])):
                        mask = cv2.fillPoly(mask, [np.array(cate[c][j][0])], cate_color)
                cv2.imwrite(f'../data/Dataset/damage_mask/' + c + '_val_mask/' + image_name.replace('.jpg', '') + '.png', cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY))
                cate[c] = []

loading annotations into memory...
Done (t=0.72s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


100%|██████████████████████████████████████████████████████████████████████████████| 1685/1685 [01:18<00:00, 21.54it/s]


loading annotations into memory...
Done (t=0.54s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


100%|██████████████████████████████████████████████████████████████████████████████| 1631/1631 [00:49<00:00, 32.72it/s]


loading annotations into memory...
Done (t=1.67s)
creating index...
index created!
loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
loading annotations into memory...
Done (t=0.67s)
creating index...
index created!


100%|██████████████████████████████████████████████████████████████████████████████| 5937/5937 [05:51<00:00, 16.87it/s]


loading annotations into memory...
Done (t=2.69s)
creating index...
index created!
loading annotations into memory...
Done (t=0.62s)
creating index...
index created!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


100%|██████████████████████████████████████████████████████████████████████████████| 2240/2240 [00:38<00:00, 58.55it/s]


In [2]:
!pip install pycocotools

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp39-cp39-win_amd64.whl size=77744 sha256=8d7be82d669a60760ff37b44247f982a32521126fb0fb0016627c4eb8991f0b4
  Stored in directory: c:\users\shs03\appdata\local\pip\cache\wheels\2f\58\25\e78f1f766e904a9071266661d20d0bc6644df86bcd160aba11
Successfully built pycocotools


In [ ]:
import json
with open('./data/datainfo/damage_Breakage_train.json', 'r') as f:
        part = json.load(f)

len(part['images'])


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image

part_json = './data/datainfo/damage_Breakage_train.json'

coco = COCO(part_json)

img_ids = coco.getImgIds()

for i in range(0, len(part['images'])):
        image_id = int(img_ids[i])

        image_infos = coco.loadImgs(image_id)[0]
        width = image_infos['width']
        height = image_infos['height']
        

        image_seg = coco.loadAnns(image_id)[0]['segmentation']
        mask = np.zeros((width,height, 3),dtype = np.uint8)
        tmp_list =[]
        for i in range(len(image_seg[0][0])):
               tmp_list.append(np.array(image_seg[0][0][i])) 
        
        print(tmp_list)
        cv2.fillPoly(mask,tmp_list, (0,0,255))
        cv2.imwrite(f'{image_id}.png', mask)